In [121]:
!pip install openpyxl
!pip install mlxtend
!pip install xlsxwriter 

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
                                              0.0/154.8 kB ? eta -:--:--
     -------                                 30.7/154.8 kB 1.4 MB/s eta 0:00:01
     ---------------------------            112.6/154.8 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 154.8/154.8 kB 1.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [198]:
import pandas as pd
from mlxtend.frequent_patterns import association_rules, apriori
import time

In [3]:
df = pd.read_excel('交易資料集.xlsx')
df.head()

,INVOICE_NO,CUST_ID,ITEM_ID,ITEM_NO,PRODUCT_TYPE,TRX_DATE,QUANTITY
0,CX47348203,3218,3217532,M25P40-VMN6TPB,MEMORY_EMBEDED,2016-07-26,2500
1,CX47346522,2470,3326781,AU80610006237AASLBX9,CPU / MPU,2016-07-11,50
2,CX47348534,16135,740487,MMBD2837LT1G,DISCRETE,2016-07-27,3000
3,A20160700174,999999999,3434776,IHLP1616ABER2R2M11,PEMCO,2016-07-29,0
4,CX47346184,2356,70072,MMBT3906LT1G,DISCRETE,2016-07-06,12000


In [5]:
df.shape

(157396, 7)

In [6]:
# remove rows where QUANTITY is less or equal to zero
df = df[df.QUANTITY > 0]

In [7]:
df

,INVOICE_NO,CUST_ID,ITEM_ID,ITEM_NO,PRODUCT_TYPE,TRX_DATE,QUANTITY
0,CX47348203,3218,3217532,M25P40-VMN6TPB,MEMORY_EMBEDED,2016-07-26,2500
1,CX47346522,2470,3326781,AU80610006237AASLBX9,CPU / MPU,2016-07-11,50
2,CX47348534,16135,740487,MMBD2837LT1G,DISCRETE,2016-07-27,3000
4,CX47346184,2356,70072,MMBT3906LT1G,DISCRETE,2016-07-06,12000
6,CX47347899,10228,3420352,TMP103AYFFR,LINEAR IC,2016-07-25,3000
...,...,...,...,...,...,...,...
157389,216071271,2100,14691206,"74HCT365D,653",LOGIC IC,2016-07-06,7030
157390,216075314,2197,3249493,"74HC04D,653",LOGIC IC,2016-07-21,347
157391,216072287,2579,133619,"74LVC2G17GW,125",LOGIC IC,2016-07-11,1644
157392,216077517,43262,15231095,MT41K256M16TW-107:P TRAY,MEMORY_EMBEDED,2016-07-28,300000


In [8]:
# INVOICE_NO: sum of transaction
# ITEM_ID: sum of type of items
df.nunique()

INVOICE_NO      36498
CUST_ID          4273
ITEM_ID         23710
ITEM_NO         23710
PRODUCT_TYPE       10
TRX_DATE           36
QUANTITY         6479
dtype: int64

In [9]:
df2 = df[["INVOICE_NO", "ITEM_ID"]]
df2

,INVOICE_NO,ITEM_ID
0,CX47348203,3217532
1,CX47346522,3326781
2,CX47348534,740487
4,CX47346184,70072
6,CX47347899,3420352
...,...,...
157389,216071271,14691206
157390,216075314,3249493
157391,216072287,133619
157392,216077517,15231095


In [10]:
# set of items of transactions containing tid
h_data = {}
items = []
#set of indices of transactions containing item
v_data = {}
for i in df.index:
    if df['ITEM_ID'][i] not in items:
        items.append(df['ITEM_ID'][i])
    if str(df['INVOICE_NO'][i]) in h_data:
        h_data[str(df['INVOICE_NO'][i])].append(str(df['ITEM_ID'][i]))
    else:
        h_data[str(df['INVOICE_NO'][i])] = [str(df['ITEM_ID'][i])]
        
    if str(df['ITEM_ID'][i]) in v_data:
        v_data[str(df['ITEM_ID'][i])].append(str(df['INVOICE_NO'][i]))
    else:
        v_data[str(df['ITEM_ID'][i])] = [str(df['INVOICE_NO'][i])]


In [14]:
class UtilityList:
    def __init__(self, item, UL):
        self.item = item
        self.UL = UL
        pass

In [15]:
def Construct(P_UL, Px_UL, Py_UL):
    Pxy_UL = []
    Ex_index = 0
    Ey_index = 0
    while Ex_index < len(Px_UL) and Ey_index < len(Py_UL):
        if Px_UL[Ex_index] == Py_UL[Ey_index]:
            Exy = []
            if P_UL is not None:
                for E in range(len(P_UL)):
                    if P_UL[E] == Px_UL[Ex_index]:
                        Exy = [Px_UL[Ex_index]]
            else:
                Exy = [Px_UL[Ex_index]]
            Pxy_UL.append(Exy)
            Ex_index += 1
            Ey_index += 1
        else:
            if Px_UL[Ex_index] > Py_UL[Ey_index]:
                Ey_index += 1
            else:
                Ex_index += 1
    return Pxy_UL


In [16]:
# brute force
def Construct2(P_UL, Px_UL, Py_UL):
    Pxy_UL = []
    for i in Px_UL:
        for j in Py_UL:
            if i == j:
                Pxy_UL.append(i)
    return Pxy_UL


In [182]:
def HUI_Miner(P, ULs, minutil):
    global frequent_itemsets
    for X in ULs:
        if len(X.UL) >= minutil:
            frequent_itemsets.append([len(X.UL)/len(h_data.keys()), [X.item[0]]])
#             print(X.item, len(X.UL)/len(h_data.keys()))
            exULs = []
            for Y in ULs[ULs.index(X)+1:]:   
                X.item + [Y.item[-1]]
                exULs.append(UtilityList(X.item + [Y.item[-1]], Construct2(P.UL, X.UL, Y.UL)))
            HUI_Miner(X, exULs, minutil)


In [23]:
nn = []
for k, v in v_data.items():
    nn.append(UtilityList([k], list(set(v_data[k]))))

In [183]:
# frequent itemset by item merge pproach
frequent_itemsets = []
HUI_Miner(UtilityList('', None), ULs=nn, minutil=len(h_data.keys())*0.001)

In [184]:
# frequent itemset count
len(frequent_itemsets)

281

In [185]:
frequent_itemsets

[[0.003178256342813305, ['70072']],
 [0.0011507479861910242, ['70072']],
 [0.0012055455093429777, ['3420352']],
 [0.0011507479861910242, ['135739']],
 [0.0029864650117814675, ['149674']],
 [0.003370047673845142, ['70057']],
 [0.0020549071181982575, ['3223014']],
 [0.0060003287851389114, ['70415']],
 [0.0017809195024384899, ['15471068']],
 [0.0027124773960217, ['14819946']],
 [0.0013151405556468849, ['14790349']],
 [0.001643925694558606, ['14671860']],
 [0.0010959504630390707, ['14671860']],
 [0.0010685517014630938, ['14671860']],
 [0.0012603430324949311, ['15181053']],
 [0.0010411529398871172, ['15201033']],
 [0.0018905145487423968, ['15192100']],
 [0.0011507479861910242, ['15192100']],
 [0.0011233492246150473, ['15192100']],
 [0.0010411529398871172, ['15192100']],
 [0.0010411529398871172, ['15452141']],
 [0.0018357170255904433, ['15455982']],
 [0.0011507479861910242, ['14921004']],
 [0.0015069318866787222, ['14481833']],
 [0.0011233492246150473, ['14481833']],
 [0.0010685517014630938,

In [187]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(list(h_data.values())).transform(list(h_data.values()))
df2 = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df2, min_support=0.001, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

In [188]:
frequent_itemsets

,support,itemsets
0,0.001480,(111064)
1,0.001014,(12560591)
2,0.002028,(125741)
3,0.001014,(125765)
4,0.001206,(126109)
...,...,...
276,0.001233,"(70509, 88411, 1697)"
277,0.001041,"(70509, 1698, 23004)"
278,0.001315,"(70509, 88411, 1698)"
279,0.001041,"(70509, 88444, 1708)"


In [148]:
print(rules)

         antecedents consequents  antecedent support  consequent support  \
0           (135492)     (70509)            0.001260            0.008137   
1           (135492)     (88411)            0.001260            0.004576   
2           (135493)     (70509)            0.001671            0.008137   
3           (135493)     (88411)            0.001671            0.004576   
4         (14481833)  (15157064)            0.001507            0.002055   
5         (14675955)  (15192100)            0.001452            0.001891   
6         (14760944)  (14752201)            0.001781            0.001562   
7         (14752201)  (14760944)            0.001562            0.001781   
8         (14876857)  (14870535)            0.001123            0.001096   
9         (14870535)  (14876857)            0.001096            0.001123   
10        (14980086)  (15192100)            0.001589            0.001891   
11        (15194056)  (15192100)            0.001452            0.001891   
12          

In [149]:
rules['antecedents']

0             (135492)
1             (135492)
2             (135493)
3             (135493)
4           (14481833)
5           (14675955)
6           (14760944)
7           (14752201)
8           (14876857)
9           (14870535)
10          (14980086)
11          (15194056)
12             (88676)
13             (88622)
14     (70509, 135493)
15     (135493, 88411)
16       (70509, 1697)
17       (88411, 1697)
18       (1698, 23004)
19       (88411, 1698)
20       (88444, 1708)
21    (70509, 3336149)
22    (3336149, 88411)
Name: antecedents, dtype: object

In [150]:
rules["consequents"]

0        (70509)
1        (88411)
2        (70509)
3        (88411)
4     (15157064)
5     (15192100)
6     (14752201)
7     (14760944)
8     (14870535)
9     (14876857)
10    (15192100)
11    (15192100)
12       (88411)
13       (88623)
14       (88411)
15       (70509)
16       (88411)
17       (70509)
18       (70509)
19       (70509)
20       (70509)
21       (88411)
22       (70509)
Name: consequents, dtype: object

In [151]:
rules["antecedents"] = rules["antecedents"].apply(lambda x: ', '.join(list(x))).astype("unicode")
rules["consequents"] = rules["consequents"].apply(lambda x: ', '.join(list(x))).astype("unicode")

In [160]:
# write file
import xlsxwriter
workbook = xlsxwriter.Workbook('rule.xlsx')
worksheet = workbook.add_worksheet()
worksheet.write(0, 0, "antecedents")
worksheet.write(0, 1, "consequents")
row = 1
for i, j in zip(rules['antecedents'], rules['consequents']):
    worksheet.write(row, 0, i)
    worksheet.write(row, 1, j)
    row+=1
    
workbook.close()

In [161]:
# read file
df_rule = pd.read_excel('rule.xlsx')
df_rule.head()

,antecedents,consequents
0,135492,70509
1,135492,88411
2,135493,70509
3,135493,88411
4,14481833,15157064


In [186]:
# input product to find the right hand side itemsets
def find_consequents(antecedents, df):
    li = []
    for k, v in zip(df_rule["antecedents"], df_rule["consequents"]):
        if antecedents in k:
            li.append(v)
    return list(set(li))
    

find_consequents("3336149, 88411", df_rule)

[70509]

In [199]:
# support count versis rules base on confidence=0.7, 
min_support = [0.001, 0.0011, 0.0012, 0.0013, 0.0014]
rules_count = []
elapsed_time = []
for i in min_support:
    start_time = time.time()
    te = TransactionEncoder()
    te_ary = te.fit(list(h_data.values())).transform(list(h_data.values()))
    df2 = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = apriori(df2, min_support=i, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
    elapsed_time.append(time.time()-start_time)
    rules_count.append(len(list(rules['antecedents'])))

In [200]:
rules_count

[23, 12, 9, 3, 2]

In [201]:
# elapsed time (second)
elapsed_time

[11.58024001121521,
 7.789711236953735,
 5.9582319259643555,
 4.186016082763672,
 3.178170680999756]

In [206]:
# confidence versis rules base on support=0.001, 
confidence = [0.5, 0.55, 0.6, 0.65, 0.7]
rules_count = []
elapsed_time = []
workbook = xlsxwriter.Workbook('rule2.xlsx')
worksheet = workbook.add_worksheet()

for k in range(len(confidence)):
    start_time = time.time()
    te = TransactionEncoder()
    te_ary = te.fit(list(h_data.values())).transform(list(h_data.values()))
    df2 = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = apriori(df2, min_support=0.001, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=confidence[k])
    elapsed_time.append(time.time()-start_time)
    rules_count.append(len(list(rules['antecedents'])))
    
    
    rules["antecedents"] = rules["antecedents"].apply(lambda x: ', '.join(list(x))).astype("unicode")
    rules["consequents"] = rules["consequents"].apply(lambda x: ', '.join(list(x))).astype("unicode")
    worksheet.write(0, k*2, "antecedents"+str(confidence[k]))
    worksheet.write(0, k*2+1, "consequents"+str(confidence[k]))
    row = 1
    for i, j in zip(rules['antecedents'], rules['consequents']):
        worksheet.write(row, k*2, i)
        worksheet.write(row, k*2+1, j)
        row+=1
    
workbook.close()

In [207]:
rules_count

[57, 48, 39, 31, 23]

In [208]:
elapsed_time

[11.331083059310913,
 11.383166313171387,
 11.297866106033325,
 11.399392127990723,
 11.322789669036865]

In [232]:
# confidence versis recommendation product
df_rule2 = pd.read_excel('rule2.xlsx')
df_rule2.head()

def find_consequents2(antecedents, df, confidence):
    li = []
    for i in confidence:
        li2 = []
        for k, v in zip(df["antecedents"+str(i)].dropna(), df["consequents"+str(i)].dropna()):
            if antecedents in k:
                li2.append(v)
        li.append(list(set(li2)))
    return li

confidence = [0.5, 0.55, 0.6, 0.65, 0.7]
product_count = find_consequents2("70509", df_rule2, confidence)
print(product_count)
print("Product count:", [len(x) for x in product_count])

[['135493', '3336149', '1698', '1697', '1708', '88444', '88411', '23004'], ['1698', '1708', '1697', '88444', '88411'], ['88444', '88411', '1708', '1698'], ['88411', '1708'], [88411.0]]
Product count: [8, 5, 4, 2, 1]


In [234]:
from mlxtend.frequent_patterns import fpgrowth

frequent_itemsets = fpgrowth(df2, min_support=0.001, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

In [235]:
# support count versis rules base on confidence=0.7 by fpgrowth
min_support = [0.001, 0.0011, 0.0012, 0.0013, 0.0014]
rules_count = []
elapsed_time = []
for i in min_support:
    start_time = time.time()
    te = TransactionEncoder()
    te_ary = te.fit(list(h_data.values())).transform(list(h_data.values()))
    df2 = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = fpgrowth(df2, min_support=i, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
    elapsed_time.append(time.time()-start_time)
    rules_count.append(len(list(rules['antecedents'])))

In [236]:
# fp growth
rules_count

[23, 12, 9, 3, 2]

In [237]:
elapsed_time

[1.1289830207824707,
 1.15592360496521,
 1.2658748626708984,
 1.1400258541107178,
 1.149949073791504]

In [238]:
# confidence versis rules base on support=0.001 by fpgrowth 
confidence = [0.5, 0.55, 0.6, 0.65, 0.7]
rules_count = []
elapsed_time = []
for i in confidence:
    start_time = time.time()
    te = TransactionEncoder()
    te_ary = te.fit(list(h_data.values())).transform(list(h_data.values()))
    df2 = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = fpgrowth(df2, min_support=0.001, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=i)
    elapsed_time.append(time.time()-start_time)
    rules_count.append(len(list(rules['antecedents'])))

In [239]:
rules_count

[57, 48, 39, 31, 23]

In [240]:
elapsed_time

[1.1249439716339111,
 1.1680607795715332,
 1.1577787399291992,
 1.1121315956115723,
 1.1532478332519531]